## PROJECT GOAL:
Outbrain is a web advertising platform that displays advertisement boxes of links to pages within websites. It displays links to the sites' pages in addition to sponsored content, generating revenue from the latter.
In our project we are predicting whether the particular ad_id will be clicked or not.

This python files predict ad using random forest classifier algorithm and measure Accuracy and AUC.

## Creating a Classification Model using Random Forest Algorithm

In this file, we will implement a classification model using *Random Forest* that uses features of a Event,Document and Train dataframe to **predict the whether the ad is clicked or not** 

You should follow the steps below to build, train and test the model from the source data:

1. Build a schema of a source data for its Data Frame
2. Load the Source Data to the schema
3. Prepare the data with the features (input columns, output column as label)
4. Split the data using data.randomSplit(): Training and Testing
5. Transform the columns to a vector using VectorAssembler
6. set features and label from the vector
7. Build a ***Random Forest Algorithm*** Model with the label and features
8. Train the model
9. Prepare the testing Data Frame with features and label from the vector; Rename label to trueLabel
10. Predict and test the testing Data Frame using the model trained at the step 8
11. Compare the predicted result and trueLabel


### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier



#####The below command will start spark session when we run our file in oracle BDE. In Databricks keep this cell as False by default. But when you run file in Oracle BDE make it True.

In [5]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

### Load Source Data
The data for this project is provided as a CSV file containing details of advertisement. The data includes specific characteristics (or *features*) for each ad, as well as a *label* column indicating whether the ad was clicked or not.

You will load this data into a DataFrame and display it.

In [7]:
eventSchema = StructType([
  StructField("display_id", IntegerType(), False),
  StructField("uuid", StringType(), False),
  StructField("document_id", IntegerType(), False),
  StructField("timestamp", IntegerType(), False),
  StructField("platform", IntegerType(), False),
  StructField("geo_location", StringType(), False)
])

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, you will simply select a subset of columns to use as *features* as well as the **ArrDelay** column, which will be the *label* your model will predict.

#####Reading all necessary csv file

In [10]:
if IS_SPARK_SUBMIT_CLI:
    event = spark.read.csv('eventd.csv', inferSchema=True, header=True)
else:
    event = spark.sql("SELECT * FROM eventd_csv")

In [11]:
#create train schema
trainSchema = StructType([
  StructField("display_id", DoubleType(), False),
  StructField("DayOfWeek", DoubleType(), False),
  StructField("clicked", IntegerType(), False)
])

In [12]:
#Read traind.csv file
if IS_SPARK_SUBMIT_CLI:
    train = spark.read.csv('traind.csv', inferSchema=True, header=True)
else:
    train = spark.sql("SELECT * FROM traind_csv")

In [13]:
# create Promoted_Content Schema
promoted_contentSchema = StructType([
  StructField("ad_id", IntegerType(), False),
  StructField("document_id", IntegerType(), False),
  StructField("campaign", IntegerType(), False),
  StructField("advertiser_id", IntegerType(), False)
])

In [14]:
# Read Promoted_content.csv file
if IS_SPARK_SUBMIT_CLI:
    promoted_content = spark.read.csv('promoted_content.csv', inferSchema=True, header=True)
else:
    promoted_content = spark.sql("SELECT * FROM promoted_content_csv")

Below command will join event and train file using disply_id as primary key. It will also drop display_id from event table and ad_id from train table as it create ambiguity error. Then it will assign result to new dataframe data1.

In [16]:
data1=event.join(train,event.display_id==train.display_id).drop(event.display_id).drop(train.ad_id)

Following cell will join two dataframe data1 (which created from joining of event and train table) with Promoted content dataframe using document_id as primary key. Then it will assign result to new dataframe data2.

In [18]:
data2=data1.join(promoted_content,data1.document_id==promoted_content.document_id).drop(data1.document_id)

Following table select disply_id,document_id,platform,ad_id,camaign_id,advertiser_id which are features which help us to train our dataset and clicked column which is label column. Selected column will assign to new dataframe called data.

In [20]:
data =  data2.select(col("display_id").cast(DoubleType()), col("document_id").cast(DoubleType()), col("platform").cast(DoubleType()), col("ad_id").cast(DoubleType()), col("campaign_id").cast(DoubleType()),col("advertiser_id").cast(DoubleType()),col("clicked").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [22]:
splits = data.randomSplit([0.7, 0.3])     
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Prepare the Training Data create Pipeline
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector,Normlize data using Minmax.

In [24]:
# Import RandomClassifier Algorithm 
from pyspark.ml.classification import RandomForestClassifier

# Convert following feature column in one vector for train data
assembler = VectorAssembler(inputCols = ["display_id", "document_id", "platform", "ad_id","campaign_id","advertiser_id"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols = ["clicked"],outputCol="label")

#Normlize feature data 
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")

#Convert Normlize feature data to vector
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")

#following Random forest algorithm train the classifiction model
dt = RandomForestClassifier(labelCol="label", featuresCol="features",impurity="gini",featureSubsetStrategy="auto",numTrees=10,maxDepth=30,maxBins=128,seed=1234)

# Following command will create pipeline with different stages
pipeline = Pipeline(stages=[assembler,minMax,featVect,dt])

The pipeline itself is an estimator, and so it has a fit method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model.

In [26]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!

### Test the Recommender 
The model produced by the pipeline is a transformed that will apply to all stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the test DataFrame using the pipeline to generate label prediction

In [28]:
# piplineModel with train data set applies test data set and generate predictions
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+----------------------------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+----------------------------------------------------------------------------------------------------------+----------+---------+
[0.23366817877427618,0.5014688106399997,0.0,0.3169388015574543,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.23366817877427618,0.5014688106399997,0.0,0.31694054993452325,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.23366817877427618,0.5014688106399997,0.0,0.31694229831159226,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.23366817877427618,0.5014688106399997,0.0,0.31694579506573023,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.889238349979095,0.9782809714595208,0.5,0.9464909198036923,0.006885458701357338,2.838489923360772E-4] |0.0 |0 |
[0.10106117287015641,0.2753508825417777,0.5,0.07751779410761961,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.10106117287015641,0.2753508825417777,0.5,0.07751954248468859,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.10106117287015641,0.2753508825417777,0.5,0.07752129086175757,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.10106117287015641,0.2753508825417777,0.5,0.07752478761589554,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.2753508825417777,0.5,0.07751954248468859,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.2753508825417777,0.5,0.07752129086175757,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.2753508825417777,0.5,0.07752303923882656,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.2753508825417777,0.5,0.07752653599296454,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.2753508825417777,0.5,0.07842520180642319,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.41789857287885424,0.12229902043422275,1.0,0.008285558929923298,0.03036374410926433,0.01845018450184502]|0.0 |0 |
[0.524338853152041,0.2753508825417777,0.0,0.07751779410761961,0.14343764991393176,0.01107011070110701] |0.0 |1 |
[0.524338853152041,0.2753508825417777,0.0,0.07752828437003352,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.5587251757521771,0.2753508825417777,1.0,0.07751779410761961,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.5587251757521771,0.2753508825417777,1.0,0.07752303923882656,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5587251757521771,0.2753508825417777,1.0,0.07842520180642319,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.6057386124259493,0.2753508825417777,1.0,0.07751779410761961,0.14343764991393176,0.01107011070110701] |1.0 |1 |
[0.6057386124259493,0.2753508825417777,1.0,0.07752303923882656,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.6057386124259493,0.2753508825417777,1.0,0.07752653599296454,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.9036495805567077,0.2753508825417777,1.0,0.07752653599296454,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.2753508825417777,1.0,0.07752828437003352,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.2753508825417777,1.0,0.07842520180642319,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.2753508825417777,1.0,0.07752129086175757,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.2753508825417777,1.0,0.07752653599296454,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9411250453208072,0.48387751661603245,0.0,0.26714502263274115,0.13502836018850353,0.4175418677263696] |0.0 |0 |
[0.018739304630639945,0.04201659271799912,0.0,0.36759977550838435,0.6799108276659989,0.019869429463525403]|0.0 |0 |
[0.244645084459711,0.6956470119836297,1.0,0.6700777503282578,0.02342184722183029,0.04598353675844451] |0.0 |0 |
[0.5989180200513312,0.816205698052705,0.0,0.813614262560778,0.006885458701357338,2.838489923360772E-4] |0.0 |0 |
[0.9282464901330129,0.963573854811924,0.5,0.940438

Following cell measure accuracy of Algorithm using MultiClassificationEvaluater and evaluate using predicted data.

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print(accuracy)

0.9520460358056266

Following Code will give total Error in project

In [32]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.047954

#### Random Forest Evaluator
Calculate AUC using Random Forest Evaluator.

In [34]:
rf_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
rf_auc = rf_evaluator.evaluate(prediction)
print("AUC for Random Forest is= ", rf_auc)

AUC for Random Forest is= 0.9499206293049177

#### Random Forest Confusion matrix
Calculate confusion matrix and measure precision,Recall

In [36]:
# Only for Classification Logistic Regression not for Linear Regression

tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()



+---------+------------------+
 metric| value|
+---------+------------------+
 TP| 1562.0|
 FP| 138.0|
 TN| 11839.0|
 FN| 537.0|
Precision|0.9188235294117647|
 Recall|0.7441638875655073|
+---------+------------------+